In [45]:
seq_list = {}
f_fa = open('insulin_pep.uniprot.fa','r')
for line in f_fa:
    if line.startswith('>'):
        seq_h = line.strip()
        seq_list[seq_h] = ''
    else:
        seq_list[seq_h] += line.strip()
f_fa.close()

human_insulin = seq_list['>sp|P01308|INS_HUMAN Insulin OS=Homo sapiens GN=INS PE=1 SV=1']

a = 0
for key in seq_list.keys():
    points = []
    query = seq_list[key]
    k = len(human_insulin)-len(query)
    os = key.split()[2:4]
    name = os[0][3:] + ' ' + os[1]
    if key == '>sp|P01308|INS_HUMAN Insulin OS=Homo sapiens GN=INS PE=1 SV=1':
        continue
    for j in range(0,k+1):
        for i in range(len(query)):
            if query[i] == human_insulin[i+j]:
                a += 1
        points.append(a)
        a = 0
        
    print ('%s - best match score %d'%(name,max(points)))

Mus musculus - best match score 58
Xenopus laevis - best match score 36
Danio rerio - best match score 28
Bos taurus - best match score 66
Sus scrofa - best match score 64
Gallus gallus - best match score 50
